In [1]:
from selenium import webdriver
from time import sleep
import config_essentials
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

driver=webdriver.Chrome(executable_path=config_essentials.CHROMELOC)


C:\Users\elenc\AppData\Local\Temp/ipykernel_7400/511810772.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(executable_path=config_essentials.CHROMELOC)


In [6]:
# head over to garage and moving sales
driver.get("https://gab.com/auth/sign_in")

In [7]:
login_form = driver.find_element_by_id("user_email").send_keys(config_essentials.UNAME)
login_form = driver.find_element_by_id("user_password").send_keys(config_essentials.PWORD)
sleep(1)
try_click = driver.find_element_by_name("button").click()

C:\Users\elenc\AppData\Local\Temp/ipykernel_7400/3259480125.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  login_form = driver.find_element_by_id("user_email").send_keys(config_essentials.UNAME)
C:\Users\elenc\AppData\Local\Temp/ipykernel_7400/3259480125.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  login_form = driver.find_element_by_id("user_password").send_keys(config_essentials.PWORD)
C:\Users\elenc\AppData\Local\Temp/ipykernel_7400/3259480125.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  try_click = driver.find_element_by_name("button").click()


In [14]:
# I am only in the 'science' group that has 90k members so head to my groups feed
driver.get("https://gab.com/groups")

In [15]:
# scroll down 5x to load many more entries
for _ in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(5)

In [48]:
# the plan is to search for modal verbs such as: might, may, could
# these terms are a bti fo a sensational practice since 'might not' and 'could not' are equally reasonable framings
# science journalism (often sensationalism) prefers tehse terms far moreso over science

modal_verbs  = ['possibly', 'can', 'could', 'may', 'might', 'should']

# I am specificalyl interested in articles linked to and will be mining the summary box for posted links
# the question - does gab/science disucss certain things or sensational-ish things

bs = BeautifulSoup(driver.page_source, 'html.parser')

# grab the news feed column
block = bs.find(class_="_UuSG")

def check_link_preview_titles(search_terms, ret_dict={"TITLE":[], "TERMS":[]}):
    
    # if I do not clear the default dict it will accumulate values between calls
    ret_dict={"TITLE":[], "TERMS":[]}
    
    # use the super bizarre span class for link preview titles to gather them up
    for listing in block.find_all('span',{'class':'_UuSG _3rXHO _3_54N _1-fAn _2ZzNB _3hcKE _3aRhh'}):
        found = []
        ltext = listing.text.lower().strip()
        for term in search_terms:
            if re.search(r'\b{}\b'.format(term), ltext):
                found.append(term)
        if len(found)>0:
            ret_dict["TITLE"].append(ltext)
            ret_dict["TERMS"].append(" ".join(found))
    
    return ret_dict

# search for modal verbs often reveals few ...
evidence = check_link_preview_titles(modal_verbs)
df       = pd.DataFrame.from_dict(evidence)
df

,TITLE,TERMS
0,could a molecule found in broccoli help the br...,could


In [49]:
# that did not reveal much at all.  let's look for pandemic words
pandemic     = ['covid-19','covid','pandemic','quarantine']
covid_terms  = check_link_preview_titles(pandemic)  
df2          = pd.DataFrame.from_dict(covid_terms)
df2

# well, this worked.  when will this topic become less popular?

,TITLE,TERMS
0,pre-infection 25-hydroxyvitamin d3 levels and ...,covid-19 covid
1,"lockdowns, school closures and limiting gather...",covid
2,ivermectin shows ‘antiviral effect’ against co...,covid
3,thrive time show: covid vaccines part of bigge...,covid
4,austrian government challenged to either prove...,covid pandemic
5,thousands take to the streets in protest of co...,covid-19 covid
6,yale now asking students to spy on each other ...,covid
7,study confirms that vitamin d significantly re...,covid-19 covid
8,lawyer thomas renz accuses military of covid c...,covid
9,viral video shows increasing number of illness...,covid


In [5]:
df2.to_csv('pandemic nuttiness.txt')
driver.quit()